In [3]:
from bs4 import BeautifulSoup
import requests

import sys
import time
import re
import urllib.request


In [2]:
chromedriver_dir = '/Users/mongkyo/Projects/kakao-chatbot/crawling/chromedriver'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# driver = webdriver.Chrome(chormedriver_dir, chrome_option=options)

In [6]:
keyword = '82년생김지영'
res = requests.get(f'https://nsulib.nsu.ac.kr/search/tot/result?st=KWRD&si=TOTAL&q={keyword}&x=0&y=0')
html = res.text
soup = BeautifulSoup(html, 'html.parser')
book_info = []
ul = soup.select_one('.resultList')


IndexError: list index out of range

In [12]:
ul.select('.items')

[<li class="items" id="item_CATTOT000000469834">
 <dl>
 <dt class="title">선택</dt>
 <dd class="dataCheck"><input name="data" title="선택" type="checkbox" value="CAT000000469834"/></dd>
 <dt class="title">No</dt>
 <dd class="num">1.</dd>
 <dt class="title">표지이미지</dt>
 <dd class="book">
 <a href="/search/detail/CATTOT000000469834?mainLink=/search/tot?mId=101020100&amp;briefLink=/search/tot/result?q=82%EB%85%84%EC%83%9D%EA%B9%80%EC%A7%80%EC%98%81_A_st=KWRD_A_y=0_A_x=0_A_si=TOTAL">
 <img alt="표지이미지" height="103" id="bookImg_CATTOT000000469834" src="/image/ko/solution/common/ico/type_m_M.png" width="75"/>
 </a>
 <p class="selectBtn">
 <a class="cartBtn" href="javascript:toggleBasket('CAT000000469834');" id="basket_CAT000000469834" title="바구니 담기">바구니 담기</a>
 <a class="myLibBtn" href="javascript:insertMyLib('CAT000000469834');" id="myLib_CAT000000469834" title="내서재에 담기">내서재에 담기</a>
 </p>
 </dd>
 <dt class="title">서명</dt>
 <dd class="title">
 <a href="/search/detail/CATTOT000000469834?mainLink=/s

In [15]:
li = ul.select('.items')[0]
book_one = dict()
book_detail = dict()
book_one['title'] = li.select_one('dd.title > a').get_text()
book_one['author'] = li.select('dd.info')[0].get_text()
book_one['publisher'] = li.select('dd.info')[1].get_text()
book_one['publish_year'] = li.select('dd.info')[2].get_text()
test_text = li.get('id')
no = re.search(r'(CAT\w+\d+)', test_text).group()
detail_html = requests.get(f'https://nsulib.nsu.ac.kr/search/detail/{no}').text
detail_soup = BeautifulSoup(detail_html, 'html.parser')
div_li = detail_soup.select_one('.listTable > table')
book_detail['no'] = div_li.select_one('td.num').get_text()
book_detail['callNum'] = div_li.select_one('td.callNum').get_text()
book_detail['location'] = div_li.select_one('td.location').get_text(" ", strip=True)
book_detail['status'] = div_li.select_one('span.status').get_text()
book_one['detail'] = book_detail
book_info.append(book_one)
book_one

{'title': '82년생 김지영 : 조남주 장편소설',
 'author': '조남주',
 'publisher': '민음사',
 'publish_year': '2016',
 'detail': {'no': '1',
  'callNum': '추천 813.7 조192ㅍ',
  'location': '중앙도서관/남서울대추천자료실(5층)/',
  'status': '대출중'}}

## 미완성1 - 크롤링 완성인줄 알았음

In [3]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_id('type1q')  # 학교 홈페이지 검색창
elem.send_keys('검은집')  # 이 부분에 검색하고 싶은 도서 이름을 넣어주면 된다
elem.send_keys(Keys.RETURN)
a_tag = driver.find_elements_by_xpath('//a[contains(@href,"#previewLocation")]')
for a in a_tag[:3+1]:
    a.click()
time.sleep(2)  # 클릭하고 html코드를 가져오는 과정이 너무 빨라서 클릭한 내용을 적용시키지 못하고 바로 긁어오기 때문에 html코드를 불러올 1초정도의 시간을 주는 것
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
ul = soup.select_one('.resultList')
book_info = []
for li in ul.select('.items'):
    book_one = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = soup.find('tbody')
    for tr in tbody.find_all('tr'):
        book_tmp = dict()
        no = tr.select_one('.footable-first-column').get_text()
        book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
        book_tmp['location'] = tr.select_one('.location').get_text()
        book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
        book_tmp['status'] = tr.select_one('.status').get_text()
        book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
        book_one[f'{no}'] = book_tmp
    print(book_one)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=72.0.3626.96)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.6 x86_64)


# 완성

In [6]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_id('type1q')  # 학교 홈페이지 검색창
elem.send_keys('검은집')  # 이 부분에 검색하고 싶은 도서 이름을 넣어주면 된다
elem.send_keys(Keys.RETURN)
a_tag = driver.find_elements_by_xpath('//a[contains(@href,"#previewLocation")]')
for a in a_tag[:3+1]:
    time.sleep(1)
    a.click()
time.sleep(2)  # 클릭하고 html코드를 가져오는 과정이 너무 빨라서 클릭한 내용을 적용시키지 못하고 바로 긁어오기 때문에 html코드를 불러올 1초정도의 시간을 주는 것
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
ul = soup.select_one('.resultList')
book_info = []
for i in range(0, 2+1):
    li = ul.select('.items')[i]
    book_one = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = li.select_one('tbody')
    for tr in tbody.find_all('tr'):
        book_tmp = dict()
        no = tr.select_one('.footable-first-column').get_text()
        book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
        book_tmp['location'] = tr.select_one('.location').get_text()
        book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
        book_tmp['status'] = tr.select_one('.status').get_text()
        book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
        book_one[f'{no}'] = book_tmp
    print(book_one)

{'title': '검은 집', 'author': '기시 유스케', 'publisher': '창해', 'publish_year': '2004', '1': {'no': '1', 'location': '중앙도서관/동양서 제2자료실(6층)/', 'serial_num': '833.6 가58ㄱ', 'status': '분실(이용자분실)', 'returnDate': ''}}
{'title': '검은 집', 'author': '기시 유스케', 'publisher': '창해', 'publish_year': '2004', '1': {'no': '1', 'location': '중앙도서관/동양서 제2자료실(6층)/', 'serial_num': '833.6 기58ㄱ2', 'status': '대출가능', 'returnDate': ''}}
{'title': '검은 집', 'author': '기시 유스케', 'publisher': '창해', 'publish_year': '2013', '1': {'no': '1', 'location': '중앙도서관/동양서 제2자료실(6층)/', 'serial_num': '833.6 기58ㄱ3', 'status': '대출가능', 'returnDate': ''}}


## 아래는 크롤링 과정중에 사용했던 코드들 (지우기 아까워서 냅둠)

### 정적으로 크롤링을 시도

In [20]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_name("q")
elem.send_keys(a)
elem.send_keys(Keys.RETURN)
driver.find_element_by_xpath('//*[@id="item_CATTOT000000469834"]/dl/dd[11]/div/p/a').click()
driver.find_element_by_xpath('//*[@id="item_CATTOT000000469834"]/dl/dd[4]/a').click()
info = dict()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

TypeError: object of type 'WebElement' has no len()

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
table = soup.find('tbody')


In [32]:
for thtd in table.find_all("tr"):
    temp = thtd.th.get_text()
    if temp.replace('\n', '') == '자료유형':
        info['datatype'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '서명/저자사항':
        info['signature'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '개인저자':
        info['personal_author'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '발행사항':
        info['publication_type'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '형태사항':
        info['form_type'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '원서명':
        info['original_title'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == 'ISBN':
        info['ISBN'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '일반주기':
        info['general_note'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '서지주기':
        info['book_note'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '일반주제명':
        info['general_subject'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '분류기호':
        info['class_sign'] = thtd.td.get_text().replace('\n', '')
    elif temp.replace('\n', '') == '언어':
        info['language'] = thtd.td.get_text().replace('\n', '')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [129]:
collect_info = dict()

In [127]:
collect_table = soup.select_one('#divContent > div > div:nth-child(6) > div.searchContents > div > table > tbody')
collect_table.find_all("tr")
for tr in collect_table.find_all("tr"):
    print(tr)
    if driver.find_element_by_class_name("footable-first-column"):
        collect_info['NO'] += tr.td.get_text()

<tr class="first">
<td class="num expand footable-first-column">1</td>
<td class="accessionNo">534762</td>
<td class="callNum">813.7 조1928</td>
<td class="location">
			                    
			                        
			                        
			                        중앙도서관/동양서 제2자료실(6층)/
			                    
	                        </td>
<td>
<span class="status ing">대출중</span>
</td>
<td class="returnDate">2018-12-27</td>
<td>
</td>
<td class="service">
<div class="serviceInfo">
</div>
</td>
<td class="info footable-last-column">
<div class="serviceInfo">
</div>
</td>
</tr>
<tr>
<td class="num expand footable-first-column">2</td>
<td class="accessionNo">534763</td>
<td class="callNum">813.7 조1928 C.2</td>
<td class="location">
			                    
			                        
			                        
			                        중앙도서관/동양서 제2자료실(6층)/
			                    
	                        </td>
<td>
<span class="status available">대출가능</span>
</td>
<td class="return

## 책검색해서 그 페이지에서 책 정보 가져오기(정적) -> (동적으로 고치기)

In [34]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_name("q")  # 학교 홈페이지 검색창
elem.send_keys('정의란 무엇인가')
elem.send_keys(Keys.RETURN)
driver.find_element_by_xpath('//*[@id="item_CATTOT000000011364"]/dl/dd[11]/div/p/a').click()
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
time.sleep(1)
elem = driver.find_element_by_xpath('//*[@id="holdingW_CAT000000011364_1"]/div[2]')
html = elem.get_attribute('innerHTML')
soup = BeautifulSoup(html, 'html.parser')
tbody = soup.find('tbody')
book_info = []
for tr in tbody.find_all('tr'):
    book_tmp = dict()
    book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
    book_tmp['location'] = tr.select_one('.location').get_text()
    book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
    book_tmp['status'] = tr.select_one('.status').get_text()
    book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
    book_info.append(book_tmp)
book_info

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="item_CATTOT000000011364"]/dl/dd[11]/div/p/a"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.6 x86_64)


## 책 검색 후 여러권을 선택하고나서 책 정보들을 가져오기

In [35]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://nsulib.nsu.ac.kr/')
elem = driver.find_element_by_id('type1q')  # 학교 홈페이지 검색창
elem.send_keys('정의란 무엇인가')  # 이 부분에 검색하고 싶은 도서 이름을 넣어주면 된다
elem.send_keys(Keys.RETURN)
a_tag = driver.find_elements_by_xpath('//a[contains(@href,"#previewLocation")]')
for a in a_tag[:3+1]:
    a.click()
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
ul = soup.select_one('.resultList')
book_info = []
for li in ul.select('.items'):
    book_one = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = soup.find('tbody')
    for tr in tbody.find_all('tr'):
        book_tmp = dict()
        no = tr.select_one('.footable-first-column').get_text()
        book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
        book_tmp['location'] = tr.select_one('.location').get_text()
        book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
        book_tmp['status'] = tr.select_one('.status').get_text()
        book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
        book_one[f'{no}'] = book_tmp
    print(book_one)

AttributeError: 'NoneType' object has no attribute 'select'

In [36]:
book_one

{'title': '전산회계 2급',
 'author': '김진호',
 'publisher': '다음',
 'publish_year': '2004',
 '1': {'no': '1',
  'location': '중앙도서관/동양서 제1자료실(5층)/',
  'serial_num': '325.9 임73ㅈ',
  'status': '대출가능',
  'returnDate': ''},
 '2': {'no': '2',
  'location': '중앙도서관/동양서 제1자료실(5층)/',
  'serial_num': '325.9 임73ㅈ C.2',
  'status': '대출가능',
  'returnDate': ''}}

In [37]:
ul = soup.select_one('.resultList')
book_info = []
for li in ul.select('.items'):
    book_one = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    tbody = soup.find('tbody')
    if tbody is not None:
        for tr in tbody.find_all('tr'):
            book_tmp = dict()
            no = tr.select_one('.footable-first-column').get_text()
            book_tmp['no'] = tr.select_one('.footable-first-column').get_text()
            book_tmp['location'] = tr.select_one('.location').get_text()
            book_tmp['serial_num'] = tr.select_one('.callNum').get_text()
            book_tmp['status'] = tr.select_one('.status').get_text()
            book_tmp['returnDate'] = tr.select_one('.returnDate').get_text()
            book_one[f'{no}'] = book_tmp
    else:
        
    print(book_one)
#     if 
#     print(li)
#     print('------------------')

IndentationError: expected an indented block (<ipython-input-37-c71496e1a367>, line 23)

In [14]:
soup.select_one('mobileTable')

### dict 자료구조가 익숙하지 않아서 예제를 만들어보면서 구조를 공부함

In [118]:
book1 = {'no': '1', 'location': '중앙도서관', 'date': '2019-01-22'}
book2 = {'no': '2', 'location': '중앙도서관', 'date': ''}

In [125]:
book1['no1'] = book2
book1

{'no': '1',
 'location': '중앙도서관',
 'date': '2019-01-22',
 'no1': {'no': '2', 'location': '중앙도서관', 'date': ''}}

In [ ]:
first_dict = dict()
second_dict = dict()

In [ ]:
second_dict = {'1' : 'first',
              '2': 'second'}

In [46]:
first_dict= second_dict, third_dict

In [49]:
for i in first_dict:
    print(i)

{'1': 'first', '2': 'second'}
{'3': 'three', '4': 'four'}


In [45]:
third_dict = {'3': 'three',
             '4': 'four'}

In [42]:
book_title = dict()
book_reservation = dict()
book_reservation = {'1': '123', '2': '123333333123', '3': '234'}



In [60]:
book_info = {'datatype': '단행본',
 'signature': '82년생 김지영: 조남주 장편소설 / 조남주 [지음].',
 'personal_author': '조남주, 저',
 'publication_type': '서울 : 민음사, 2016.',
 'form_type': '190 p. ; 20cm.',
 'ISBN': '9788937473005(세트)9788937473135',
 'class_sign': '813.7 ',
 'language': '한국어',
            1 : {
                'reservation': '예약가능'
            },
             2 : {
                 'reservation': '예약불가'
             }
}

In [38]:
book_info

[]

In [56]:
info = dict
book_info = dict
book_reservation = dict

In [77]:
book_dic = ['첫번째', '두번째', '세번째', '네번째']

In [79]:
book_reset = {book_dic[0]: book_dic[1:]}

In [80]:
book_reset

{'첫번째': ['두번째', '세번째', '네번째']}

# selenium 사용 안하고 크롤링하기

##### li(items)에서 id = '000000278036 '와 같은 숫자 데이터를 뽑아야함, 그럼 detail에서 selenium없이 데이터를 가져올 수 있음

In [3]:
search_keyword = '조직행동론'
res = requests.get(f'https://nsulib.nsu.ac.kr/search/tot/result?st=KWRD&si=TOTAL&q={search_keyword}&x=0&y=0')
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [4]:
ul = soup.select_one('.resultList')
book_info = []
for i in range(0, 2+1):
    li = ul.select('.items')[i]
    book_one = dict()
    book_detail = dict()
    book_one['title'] = li.select_one('dd.title > a').get_text()
    book_one['author'] = li.select('dd.info')[0].get_text()
    book_one['publisher'] = li.select('dd.info')[1].get_text()
    book_one['publish_year'] = li.select('dd.info')[2].get_text()
    book_info.append(book_one)
    test_text = li.get('id')
    no = re.search(r'(CAT\w+\d+)', test_text).group()
    detail_html = requests.get(f'https://nsulib.nsu.ac.kr/search/detail/{no}').text
    detail_soup = BeautifulSoup(detail_html, 'html.parser')
    div_li = detail_soup.select_one('.listTable > table')
    book_detail['no'] = div_li.select_one('td.num').get_text()
    book_detail['callNum'] = div_li.select_one('td.callNum').get_text()
    book_detail['location'] = div_li.select_one('td.location').get_text(" ", strip=True)
    book_detail['status'] = div_li.select_one('span.status').get_text()
    book_one['detail'] = book_detail

In [5]:
book_info

[{'title': '조직행동론',
  'author': '김문중',
  'publisher': '청목출판사',
  'publish_year': '1998',
  'detail': {'no': '1',
   'callNum': '325.2 김36ㅈ',
   'location': '중앙도서관/동양서 제1자료실(5층)/',
   'status': '대출가능'}},
 {'title': '조직행동론',
  'author': '유기현',
  'publisher': '貿易經營社',
  'publish_year': '2003',
  'detail': {'no': '1',
   'callNum': '325.2 유18ㅈ',
   'location': '중앙도서관/동양서 제1자료실(5층)/',
   'status': '대출가능'}},
 {'title': '조직행동론 = Organizational behavior',
  'author': '이상수',
  'publisher': '형설출판사',
  'publish_year': '2004',
  'detail': {'no': '1',
   'callNum': '325.2 이52조',
   'location': '중앙도서관/동양서 제1자료실(5층)/',
   'status': '대출가능'}}]

In [35]:
i = li.select('li.items')[0]

IndexError: list index out of range

In [37]:
test_text = li.get('id')

In [40]:
re.test_text

'item_CATTOT000000278036'

In [91]:
no = re.search(r'(CAT\w+\d+)', test_text).group()

In [92]:
print(no)

CATTOT000000278036


In [93]:
detail_html = requests.get(f'https://nsulib.nsu.ac.kr/search/detail/{no}').text

In [105]:
detail_html = requests.get(f'https://nsulib.nsu.ac.kr/search/detail/{no}').text
detail_soup = BeautifulSoup(detail_html, 'html.parser')

In [129]:
div_li = detail_soup.select_one('.listTable > table')

In [130]:
div_li.select_one('td.location').get_text(" ", strip=True)

'중앙도서관/동양서 제1자료실(5층)/'

In [131]:
div_li.select_one('span.status').get_text()

'대출가능'

In [118]:
div_li

<table class="searchTable">
<caption>메세지가 없습니다</caption>
<thead>
<tr>
<th data-class="expand" scope="row">No.</th>
<th data-hide="phone" scope="row">등록번호</th>
<th data-hide="phone" scope="row">청구기호</th>
<th data-hide="expand" scope="row">소장처</th>
<th data-hide="expand" scope="row">도서상태</th>
<th data-hide="phone" scope="row">반납예정일</th>
<th scope="row">예약</th>
<th data-hide="phone" scope="row">서비스</th>
<th data-hide="phone" scope="row">매체정보</th>
</tr>
</thead>
<tbody>
<tr class="first">
<td class="num">1</td>
<td class="accessionNo">305972</td>
<td class="callNum">325.2 이52조</td>
<td class="location">
			                    
			                        
			                        
			                        중앙도서관/동양서 제1자료실(5층)/
			                    
	                        </td>
<td>
<span class="status available">대출가능</span>
</td>
<td class="returnDate"></td>
<td>
</td>
<td class="service">
<div class="serviceInfo">
<a class="service" href="/missrepo/write?accession_no=000000305972"><